In [5]:
#imported usefullibraries and chrome web driver installed
import pandas as pd
import csv
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#extract 100 links from Amazon_scraping file and store in pandas
df = pd.read_excel(r'Amazon_Scraping.xlsx')
File = pd.DataFrame(df)
web_srapper_Data = File.loc[10:110]
Asin = web_srapper_Data['Asin']
country = web_srapper_Data['country']
Asin = Asin.astype(str)
ID = web_srapper_Data['id']


List_of_products = [] 

'''get Asin and country value to create link with loop'''
for i,j in zip(country,Asin):
    
    url = "https://www.amazon."+i+"/dp/"+j
    
    # installl web driver 
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    # soup and parse the html content of amazon link
    soup =BeautifulSoup(driver.page_source, 'html.parser')
    
    # counting of div tags in not working link and working links
    results_1 = soup.find('div')
    div_len=len(results_1)
    
    #results_1 = soup.find('a')['href']
    #results_2 = soup.find('title').get_text()
    #if results_1 == '/ref=cs_404_logo/'or results_2 == ('404 - Documento non trovato'or'404 - Documento no encontrado'or'Page introuvable'):
    
    #----this section will work for the links which is working----
    if div_len <3:
     
        key = ['title','price','details','img_url']
        value = []
        Dict = {}
        soup =BeautifulSoup(driver.page_source, 'html.parser')
        
        #this code will fetch the title from the link
        title = soup.title.string
        value.append(title)#value store in the list

        ##this code will fetch the product price from the link 
        results = soup.find_all('span', class_=('a-size-base a-color-price a-color-price'))
        try:
            if results ==[]:
                results1 = soup.find('span', class_='a-size-base a-color-price')

                if results1 ==([] or None):
                    try:
                        results2 = soup.find_all('span', class_='a-color-base')
                    except:
                        results2 = soup.find('span', class_='a-color-base')
                    item2 = results2[1]
                    item3 = item2.text.strip()
                    value.append(item3)
                else:
                    item1 = results1
                    item2 = item1.text
                    value.append(item2)
            else:
                item = results
                item1 = item.text
                value.append(item1)
        except:
            results3 = soup.find('span', class_='a-size-base a-color-price a-color-price')
            item = results3
            item1 = item.text
            value.append(item1)#value store in the list

            
        ##this code will fetch the product details from the link
        soup =BeautifulSoup(driver.page_source, 'html.parser')
        try:
            results = soup.find_all('div', id='detailBullets_feature_div')
            item = results[1]
            item1 = item.text.replace('\u200e','')
            item2 = item1.replace('\n','')
            item3 = item2.replace('\u200f','')
            item4 = item3.replace('  ','')
            value.append(item4)
        except:
            exception= 'no details mentioned'
            value.append(exception) #value store in the list
           

        #this code will fetch the url of image from the link
        results = soup.find('div', id="img-canvas")
        if results == None:
            results = soup.find('div', id="imgTagWrapperId")
            item = results.children
            item1 = list(item)
            img_link = item1[1]['src']
            value.append(img_link) #value store in the list
            
        

        product_dict = dict(zip(key,value)) #zipping the dictionaries with value stored in list and keys
        Dict.update(product_dict) #updating the value in the dictionary
        List_of_products.append(Dict)# store the dictionary value in final list
        print(Dict) # printing the value to check output is right
     
    
    #----this section will work for those links are not working----
    elif div_len != 0:
        
        # declare the values
        key1 = []
        value1 = []
        Dict1 = {} 
        
        
        NA = 'not available'
        key1.append(url) # stored url which is not working
        value1.append(NA) # stored value 'not available'
        product_dict = dict(zip(key1,value1))# url and value is zipping in the dictinary
        Dict1.update(product_dict) # stored zipping value in the dictionary
        List_of_products.append(Dict1) # stored the dictionary in final list
        print(Dict1)
    else:
        # this code will work if previous code will not work if div_len value =0
        NA = 'not available'
        key1.append(url)
        value1.append(NA)
        product_dict = dict(zip(key1,value1))
        Dict1.update(product_dict)
        List_of_products.append(Dict1)
        print(Dict1)
        
#convert data into Json
        
import json

List_of_products
json_string = json.dumps(List_of_products)

# Writing to sample.json
with open("resultant.json", "w") as outfile:
    outfile.write(resultant.json)
